In [9]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import newspaper
from selenium import webdriver

In [10]:
#Settings
think_tank = "Heartland"
start_date = "2021" #Stop scraping articles that are from 2021 or older.

In [11]:
df = pd.DataFrame(columns= ["THINK_TANK", "AUTHOR", "TITLE", "CONTENT", "DATE", "LINK", "TYPE"])
df = df[["TITLE", "AUTHOR", "CONTENT", "DATE", "TYPE", "THINK_TANK", "LINK"]]
df.head()

,TITLE,AUTHOR,CONTENT,DATE,TYPE,THINK_TANK,LINK


In [12]:
#Settings for scraping libraries
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
config = newspaper.Config()
config.browser_user_agent = user_agent


In [13]:
driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")

/var/folders/px/4r2jc78x1h748q29m317jc6w0000gn/T/ipykernel_57472/1162476086.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")


In [14]:
def scrape_article(URL,type):
    try:
        article = newspaper.Article(URL,config=config)
        article.download()
        article.parse() #Download and parse article using Newspaper library

        soup = bs(article.html, 'html.parser') #Download article using Newspaper BeautifulSoup
        date = soup.find("div", class_="zn-date").text.replace("Published ", "").strip()

        if start_date in date:
            return False #Stops scraping if article older than the start_date defined earlier. 
        else:
            author = soup.find("div", class_="zn-authors").text.replace("By ", "").strip()
            title = article.title.replace(" – The Heartland Institute", "") #Deletes the generic end of the title.
            row = [title, author, article.text, date, type ,think_tank, URL]

            df.loc[len(df)] = row
            return True #Returns true to continue scraping more articles, as the articles are still within the date range 
    except:return True


In [15]:
#Run through all publication sites until start_date reached. And then the Op-ed Site. 
driver.get("https://heartland.org/publications/page/1/?topic=environment-energy") #Only environment/energy articles.
soup = bs(driver.page_source, "html.parser")

run = True #is true until the first article before start_date is reached. 

while run:
    #Scrapes all articles on the page
    publications =  soup.find_all("div", class_='zn-archive-listing')
    
    for publication in publications:
        link = publication.find("h2").find("a")["href"]
        run = scrape_article(link, "PUBLICATION")
        if run == False: 
            break
    
    #Gooes to next pages to scrape more articles there.
    next_page = soup.find("a", class_="next page-numbers")
    if next_page is None:
        run = False
    else:
        driver.get(next_page["href"])
        soup = bs(driver.page_source, "html.parser")


#Start scraping op-eds.     
driver.get("https://heartland.org/opinion/page/1/?topic=environment-energy")
soup = bs(driver.page_source, "html.parser")

run = True

while run:
    publications =  soup.find_all("div", class_='zn-archive-listing')
    for publication in publications:
        link = publication.find("h2").find("a")["href"]
        run = scrape_article(link, "OPED")
        if run == False: 
            break
    next_page = soup.find("a", class_="next page-numbers")
    if next_page is None:
        run = False
    else:
        driver.get(next_page["href"])
        soup = bs(driver.page_source, "html.parser")
    

In [17]:
#put data into csv
df.to_csv("data/heartland.csv", encoding="utf-8-sig", index=False)